## CTF (Defocus) estimation

CTF estimation has been performed using CTFFind4.

Please cite:

1. Joseph A. Mindell, Nikolaus Grigorieff.
Accurate determination of local defocus and specimen tilt in electron microscopy,
*Journal of Structural Biology* **142(3)**:334-347 (2003)

2. Alexis Rohou, Nikolaus Grigorieff.
CTFFIND4: Fast and accurate defocus estimation from electron micrographs,
*Journal of Structural Biology* **192(2)**:216-221 (2015)

In [ ]:
def get_ctffind_output_dir():
    """Get the directory where CTFFind outputs are"""
    ctffind_yaml = f"{os.getcwd()}/{proj_name}_ctffind.yaml"

    try:
        with open(ctffind_yaml, "r") as f:
            output = yaml.load(f.read(), Loader=yaml.FullLoader)
            ctffind_outdir = output["System"]["output_path"]
    except FileNotFoundError:
        print(f"{ctffind_yaml} not found, cannot find the CTFFind output directory")

    return ctffind_outdir


ctffind_outdir = get_ctffind_output_dir()

### Thon rings

Shown at 0 degrees tilt and max tilt.

In [ ]:
def get_thon_rings(ctffind_outdir):
    """Get list of filenames for 0 tilt and max tilt thon ring images"""
    thon_rings_all = {}
    thon_rings_vis = {}
    for f in os.listdir(ctffind_outdir):
        if f.endswith("ctffind.mrc"):
            ts = int(f.split("_")[1])
            if ts not in thon_rings_all.keys():
                thon_rings_all[ts] = []
            thon_rings_all[ts].append(f)
    for ts in thon_rings_all.keys():
        thon_rings = thon_rings_all[ts]
        thon_rings_vis[ts] = {}
        tas = {}
        for tr in thon_rings:
            ta = float(tr.split("_")[2])
            tas[ta] = tr
        thon_rings_vis[ts]["0.0deg"] = thon_rings[0]
        max_ta = max(tas.keys())
        thon_rings_vis[ts][f"{max_ta}deg"] = thon_rings[int(max_ta)]
    return thon_rings_vis


thon_rings = get_thon_rings(ctffind_outdir)

In [ ]:
def show_thon_rings(thon_rings_dict, ts):
    """Generate plot of thon rings at 0 and max tilt"""
    plt.figure(figsize=(5, 3))
    for i, tr in enumerate(thon_rings_dict.keys()):
        with mrcfile.open(f"{ctffind_outdir}/{thon_rings_dict[tr]}") as mrc:
            img_data = mrc.data
        plt.subplot(1, 2, i + 1)
        plt.imshow(img_data[0], cmap="Greys")
        plt.title(tr)
    plt.suptitle(ts)
    plt.tight_layout()


for ts in thon_rings.keys():
    show_thon_rings(thon_rings[ts], ts)

### CTF Resolution

In [ ]:
def get_ctf_res(ctffind_outdir):
    """Get ctf resolution from ctffind.txt files"""
    ctfres_all = {}
    for f in os.listdir(ctffind_outdir):
        if f.endswith("ctffind.txt"):
            ts = f.split("_")[1]
            if ts not in ctfres_all.keys():
                ctfres_all[ts] = {}
            with open(f"{ctffind_outdir}/{f}", "r") as textfile:
                res = float(textfile.readlines()[-1].split()[-1])
            ta = float(f.split("_")[2])
            ctfres_all[ts][ta] = res
    return ctfres_all


def plot_ctf_res(ctfres_all):
    plt.figure(figsize=(7.5, 2.5 * math.ceil(len(list(ctfres_all.keys())) / 3)))
    for i, ts in enumerate(list(ctfres_all.keys())):
        plt.subplot(math.ceil(len(list(ctfres_all.keys())) / 3), 3, i + 1)
        plt.plot(list(ctfres_all[ts].keys()), list(ctfres_all[ts].values()), ".")
        plt.xlabel("Tilt angle (deg)")
        plt.ylabel("Fit resolution (A)")
        plt.title(ts)
    plt.tight_layout()


ctfres_all = get_ctf_res(ctffind_outdir)
plot_ctf_res(ctfres_all)